In [1]:
!pip install rdkit RDKit 

In [1]:
!wget -q -O vina https:// github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.3/vina_1.2.3_linux_x86_64 -P /usr/local/bin/

'wget' is not recognized as an internal or external command,
operable program or batch file.


## Prepare Ligand

ligands were gathered from the URL below with some modifications to the smiles. The modification made was by converting the ligand so that they only have one chain. For example for the 2nd ligand, the original smiles was 'CN1CCCC(CC2c3ccccc3Sc3ccccc32)C1.Cl.O' which shows that there are additional Hcl and H2O on the strucutr which is not part of the backbone, this will result to an error later when docking with vina.

https://www.ebi.ac.uk/chembl/web_components/explore/compounds/STATE_ID:ENKWhnu39JuPHINAZgeTuA%3D%3D

In [3]:
from rdkit import Chem

ligands_smls = ['CC(C)NC(C)Cc1ccc(I)cc1',
              'CN1CCCC(CC2c3ccccc3Sc3ccccc32)C1',
              'CN(C)CC/C=C1\c2ccccc2CSc2ccccc21',
              'CN1CCCC(CN2c3ccccc3Sc3ccccc32)C1',
              'CC(C1=C(CCN(C)C)Cc2ccccc21)c1ccccn1',
              'CCOC(=O)[C@H](CCc1ccccc1)N[C@@H](C)C(=O)N1Cc2ccccc2C[C@H]1C(=O)O',
              'S=c1[nH]cnc2nc[nH]c12',
              'CCN(CCCCOC(=O)c1ccc(OC)c(OC)c1)C(C)Cc1ccc(OC)cc1',
              'O=C1Cc2cc(CCN3CCN(c4nsc5ccccc45)CC3)c(Cl)cc2N1',
              'CN1CCCCC1CCN1c2ccccc2Sc2ccc([S+](C)[O-])cc21',
              'Cc1ccc(O)c([C@H](CCN(C(C)C)C(C)C)c2ccccc2)c1',
              'CCCC(=O)O[C@]1(C(=O)COC(C)=O)CC[C@H]2[C@@H]3C[C@H](F)C4=CC(=O)C=C[C@]4(C)[C@@]3(F)[C@@H](O)C[C@@]21C',
              'CC(N)C12CC3CC(CC(C3)C1)C2',
              'CNCC[C@H](Oc1cccc2ccccc12)c1cccs1',
              'CCCNC(C)C(=O)Nc1ccccc1C',
              'Clc1ccc(C(Cn2ccnc2)OCc2csc3c(Cl)cccc23)c(Cl)c1',
              'COc1ccc2c(c1)[C@]13CCCC[C@@H]1[C@H](C2)N(C)CC3',
              'OC(c1cc(C(F)(F)F)nc2c(C(F)(F)F)cccc12)C1CCCCN1',
              'CCCCN(CCCC)CCC(O)c1cc2c(Cl)cc(Cl)cc2c2cc(C(F)(F)F)ccc12',
              'O=C1CN(/N=C/c2ccc(-c3ccc([N+](=O)[O-])cc3)o2)C(=O)[N-]1']

ligands_mols = []

for lig in ligands_smls:
    temp = Chem.MolFromSmiles(lig)
    ligands_mols.append(temp)

ligands_mols

<>:5: SyntaxWarning: invalid escape sequence '\c'
<>:5: SyntaxWarning: invalid escape sequence '\c'
C:\Users\hendy\AppData\Local\Temp\ipykernel_56100\416135513.py:5: SyntaxWarning: invalid escape sequence '\c'
  'CN(C)CC/C=C1\c2ccccc2CSc2ccccc21',


### Add hydrogen to atoms in all ligands

In [5]:
ligand_Hs= []

for i in range(len(ligands_mols)):
    temp = Chem.AddHs(ligands_mols[i])
    ligand_Hs.append(ligands_mols[i])

### Safe ligands to sdf files

In [7]:
ligand_Hs

In [9]:
import os

os.makedirs('docking/Ligands', exist_ok=True)

for i in range(len(ligand_Hs)):
    Chem.MolToMolFile(ligand_Hs[i], f'docking/Ligands/ligand_{i}.sdf')

# Prepare Protein

Protein used is 3KTR which consist of 2 chains, chain A being the polyadenylate binding protein 1 and chain B being ataxin 2. In this assignment, we will use only chain A as the protein. Chain B will be used to find the docking site at later stage of the assignment.

In [13]:
import MDAnalysis as mda

u= mda.Universe('3ktr.pdb')
protein = u.select_atoms('protein and chainID A')
protein

C:\Users\hendy\anaconda3\Lib\site-packages\MDAnalysis\topology\guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: CD
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))


<AtomGroup with 618 atoms>

### save the protein as a file

In [15]:
os.makedirs('docking/protein', exist_ok= True)
protein.write('docking/protein/protein_3ktr.pdb')

C:\Users\hendy\anaconda3\Lib\site-packages\MDAnalysis\coordinates\PDB.py:1153: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"


### Fixing protein structure with pdb2pqr

In [17]:
!pip install pdb2pqr

In [19]:
!pdb2pqr --pdb-output=docking/protein/protein_h.pdb --pH=7.4 docking/protein/protein_3ktr.pdb docking/protein/protein_3ktr.pqr --whitespace

INFO:PDB2PQR v3.6.2: biomolecular structure conversion software.
INFO:Please cite:  Jurrus E, et al.  Improvements to the APBS biomolecular solvation software suite.  Protein Sci 27 112-128 (2018).
INFO:Please cite:  Dolinsky TJ, et al.  PDB2PQR: expanding and upgrading automated preparation of biomolecular structures for molecular simulations. Nucleic Acids Res 35 W522-W525 (2007).
INFO:Checking and transforming input arguments.
INFO:Loading topology files.
INFO:Loading molecule: docking/protein/protein_3ktr.pdb
ERROR:Error parsing line: invalid literal for int() with base 10: ''
ERROR:<REMARK     2>
ERROR:Truncating remaining errors for record type:REMARK

ERROR:['REMARK']
INFO:Setting up molecule.
INFO:Created biomolecule object with 81 residues and 610 atoms.
INFO:Setting termini states for biomolecule chains.
INFO:Loading forcefield.
INFO:Loading hydrogen topology definitions.
INFO:Attempting to repair 1 missing atoms in biomolecule.
INFO:Added atom OXT to residue ALA A 623 at coo

### save PDB to PDBQT file

In [21]:
pdbqt_dir = 'docking/pdbqt'
os.makedirs(pdbqt_dir, exist_ok = True)

In [23]:
protein_dir = 'docking/Protein'

u= mda.Universe(f'{protein_dir}/protein_3ktr.pqr')
u.atoms.write(f'{pdbqt_dir}/3ktr.pdbqt')

C:\Users\hendy\anaconda3\Lib\site-packages\MDAnalysis\coordinates\PDBQT.py:296: UserWarning: Supplied AtomGroup was missing the following attributes: altLocs, occupancies, tempfactors. These will be written with default values. 
  warnings.warn(


In [26]:
with open(f'{pdbqt_dir}/3ktr.pdbqt', 'r') as f:
    file_content = f.read()

file_content= file_content.replace('TITLE', 'REMARK').replace('CRYST1', 'REMARK')

with open(f'{pdbqt_dir}/3ktr.pdbqt', 'w') as f:
    f.write(file_content)

### prepare ligands

In [ ]:
!pip install openbabel

In [30]:
!obabel -isdf docking/Ligands/ligand_0.sdf -opdbqt -O docking/pdbqt/ligand_0.pdbqt
!obabel -isdf docking/Ligands/ligand_1.sdf -opdbqt -O docking/pdbqt/ligand_1.pdbqt
!obabel -isdf docking/Ligands/ligand_2.sdf -opdbqt -O docking/pdbqt/ligand_2.pdbqt
!obabel -isdf docking/Ligands/ligand_3.sdf -opdbqt -O docking/pdbqt/ligand_3.pdbqt
!obabel -isdf docking/Ligands/ligand_4.sdf -opdbqt -O docking/pdbqt/ligand_4.pdbqt

!obabel -isdf docking/Ligands/ligand_5.sdf -opdbqt -O docking/pdbqt/ligand_5.pdbqt
!obabel -isdf docking/Ligands/ligand_6.sdf -opdbqt -O docking/pdbqt/ligand_6.pdbqt
!obabel -isdf docking/Ligands/ligand_7.sdf -opdbqt -O docking/pdbqt/ligand_7.pdbqt
!obabel -isdf docking/Ligands/ligand_8.sdf -opdbqt -O docking/pdbqt/ligand_8.pdbqt
!obabel -isdf docking/Ligands/ligand_9.sdf -opdbqt -O docking/pdbqt/ligand_9.pdbqt

!obabel -isdf docking/Ligands/ligand_10.sdf -opdbqt -O docking/pdbqt/ligand_10.pdbqt
!obabel -isdf docking/Ligands/ligand_11.sdf -opdbqt -O docking/pdbqt/ligand_11.pdbqt
!obabel -isdf docking/Ligands/ligand_12.sdf -opdbqt -O docking/pdbqt/ligand_12.pdbqt
!obabel -isdf docking/Ligands/ligand_13.sdf -opdbqt -O docking/pdbqt/ligand_13.pdbqt
!obabel -isdf docking/Ligands/ligand_14.sdf -opdbqt -O docking/pdbqt/ligand_14.pdbqt

!obabel -isdf docking/Ligands/ligand_15.sdf -opdbqt -O docking/pdbqt/ligand_15.pdbqt
!obabel -isdf docking/Ligands/ligand_16.sdf -opdbqt -O docking/pdbqt/ligand_16.pdbqt
!obabel -isdf docking/Ligands/ligand_17.sdf -opdbqt -O docking/pdbqt/ligand_17.pdbqt
!obabel -isdf docking/Ligands/ligand_18.sdf -opdbqt -O docking/pdbqt/ligand_18.pdbqt
!obabel -isdf docking/Ligands/ligand_19.sdf -opdbqt -O docking/pdbqt/ligand_19.pdbqt

1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted
1 molecule converted


# Docking

### find docking pocket

In [33]:
original_structure = mda.Universe('3ktr.pdb')
original_ligand = original_structure.select_atoms('protein and chainID B')

pocket_center = original_ligand.center_of_geometry()
pocket_center = pocket_center.tolist() 
pocket_center

[-14.229709902451239, 13.813389290380114, -5.012511440958003]

### define ligand box dimension

In [35]:
ligand_box = original_ligand.positions.max(axis=0) - original_ligand.positions.min(axis=0) + 5
ligand_box = ligand_box.tolist()
ligand_box

[32.20800018310547, 22.190998077392578, 33.42900085449219]

In [37]:
os.makedirs('docking/docking_results', exist_ok=True)

In [39]:
with open('docking/config.txt', 'w') as f:
    f.write(f'center_x = {pocket_center[0]}\n')
    f.write(f'center_y = {pocket_center[1]}\n')
    f.write(f'center_z = {pocket_center[2]}\n')
    f.write('\n')
    f.write(f'size_x = {ligand_box[0]}\n')
    f.write(f'size_y = {ligand_box[1]}\n')
    f.write(f'size_z = {ligand_box[2]}')

In [3]:
!vina --help


Input:
  --receptor arg        rigid part of the receptor (PDBQT)
  --flex arg            flexible side chains, if any (PDBQT)
  --ligand arg          ligand (PDBQT)

Search space (required):
  --center_x arg        X coordinate of the center
  --center_y arg        Y coordinate of the center
  --center_z arg        Z coordinate of the center
  --size_x arg          size in the X dimension (Angstroms)
  --size_y arg          size in the Y dimension (Angstroms)
  --size_z arg          size in the Z dimension (Angstroms)

Output (optional):
  --out arg             output models (PDBQT), the default is chosen based on 
                        the ligand file name
  --log arg             optionally, write log file

Misc (optional):
  --cpu arg                 the number of CPUs to use (the default is to try to
                            detect the number of CPUs or, failing that, use 1)
  --seed arg                explicit random seed
  --exhaustiveness arg (=8) exhaustiveness of the glo

In [5]:
from vina import Vina

ModuleNotFoundError: No module named 'vina'

In [13]:
!vina --version

AutoDock Vina 1.1.2 (May 11, 2011)


In [9]:
!pip install vina

  Using cached vina-1.2.5.tar.gz (89 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build vina


  error: subprocess-exited-with-error
  
  exit code: 1
  
  [67 lines of output]
  fatal: not a git repository (or any of the parent directories): .git
  Version found 1.2.5 (from __init__.py)
  running bdist_wheel
  running build
  running build_ext
  Boost library location automatically determined in this conda environment.
  Boost library location was automatically guessed at C:\Users\hendy\anaconda3\include.
  - include_dirs: ['C:\\Users\\hendy\\anaconda3\\include', 'C:\\Users\\hendy\\anaconda3\\Include', 'C:\\Users\\hendy\\anaconda3\\include', 'src/lib']
  - library_dirs: ['C:\\Users\\hendy\\anaconda3\\libs', 'C:\\Users\\hendy\\anaconda3', 'C:\\Users\\hendy\\anaconda3\\PCbuild\\amd64', 'C:\\Users\\hendy\\anaconda3\\lib']
  - swig options: ['-c++', '-small', '-naturalvar', '-fastdispatch', '-shadow', '-py3', '-IC:\\Users\\hendy\\anaconda3\\include', '-IC:\\Users\\hendy\\anaconda3\\Include', '-IC:\\Users\\hendy\\anaconda3\\include', '-Isrc/lib']
  - libraries: []
  - extra link arg

In [41]:
!vina --receptor docking/pdbqt/3ktr.pdbqt --ligand docking/pdbqt/ligand_0.pdbqt --out docking/docking_results/ligand_0_out.pdbqt --config docking/config.txt
!vina --receptor docking/pdbqt/3ktr.pdbqt --ligand docking/pdbqt/ligand_1.pdbqt --out docking/docking_results/ligand_1_out.pdbqt --config docking/config.txt
!vina --receptor docking/pdbqt/3ktr.pdbqt --ligand docking/pdbqt/ligand_2.pdbqt --out docking/docking_results/ligand_2_out.pdbqt --config docking/config.txt
!vina --receptor docking/pdbqt/3ktr.pdbqt --ligand docking/pdbqt/ligand_3.pdbqt --out docking/docking_results/ligand_3_out.pdbqt --config docking/config.txt
!vina --receptor docking/pdbqt/3ktr.pdbqt --ligand docking/pdbqt/ligand_4.pdbqt --out docking/docking_results/ligand_4_out.pdbqt --config docking/config.txt

!vina --receptor docking/pdbqt/3ktr.pdbqt --ligand docking/pdbqt/ligand_5.pdbqt --out docking/docking_results/ligand_5_out.pdbqt --config docking/config.txt
!vina --receptor docking/pdbqt/3ktr.pdbqt --ligand docking/pdbqt/ligand_6.pdbqt --out docking/docking_results/ligand_6_out.pdbqt --config docking/config.txt
!vina --receptor docking/pdbqt/3ktr.pdbqt --ligand docking/pdbqt/ligand_7.pdbqt --out docking/docking_results/ligand_7_out.pdbqt --config docking/config.txt
!vina --receptor docking/pdbqt/3ktr.pdbqt --ligand docking/pdbqt/ligand_8.pdbqt --out docking/docking_results/ligand_8_out.pdbqt --config docking/config.txt
!vina --receptor docking/pdbqt/3ktr.pdbqt --ligand docking/pdbqt/ligand_9.pdbqt --out docking/docking_results/ligand_9_out.pdbqt --config docking/config.txt

!vina --receptor docking/pdbqt/3ktr.pdbqt --ligand docking/pdbqt/ligand_10.pdbqt --out docking/docking_results/ligand_10_out.pdbqt --config docking/config.txt
!vina --receptor docking/pdbqt/3ktr.pdbqt --ligand docking/pdbqt/ligand_11.pdbqt --out docking/docking_results/ligand_11_out.pdbqt --config docking/config.txt
!vina --receptor docking/pdbqt/3ktr.pdbqt --ligand docking/pdbqt/ligand_12.pdbqt --out docking/docking_results/ligand_12_out.pdbqt --config docking/config.txt
!vina --receptor docking/pdbqt/3ktr.pdbqt --ligand docking/pdbqt/ligand_13.pdbqt --out docking/docking_results/ligand_13_out.pdbqt --config docking/config.txt
!vina --receptor docking/pdbqt/3ktr.pdbqt --ligand docking/pdbqt/ligand_14.pdbqt --out docking/docking_results/ligand_14_out.pdbqt --config docking/config.txt

!vina --receptor docking/pdbqt/3ktr.pdbqt --ligand docking/pdbqt/ligand_15.pdbqt --out docking/docking_results/ligand_15_out.pdbqt --config docking/config.txt
!vina --receptor docking/pdbqt/3ktr.pdbqt --ligand docking/pdbqt/ligand_16.pdbqt --out docking/docking_results/ligand_16_out.pdbqt --config docking/config.txt
!vina --receptor docking/pdbqt/3ktr.pdbqt --ligand docking/pdbqt/ligand_17.pdbqt --out docking/docking_results/ligand_17_out.pdbqt --config docking/config.txt
!vina --receptor docking/pdbqt/3ktr.pdbqt --ligand docking/pdbqt/ligand_18.pdbqt --out docking/docking_results/ligand_18_out.pdbqt --config docking/config.txt
!vina --receptor docking/pdbqt/3ktr.pdbqt --ligand docking/pdbqt/ligand_19.pdbqt --out docking/docking_results/ligand_19_out.pdbqt --config docking/config.txt

#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, Journal of Computational Chemistry 31 (2010)  #
# 455-461                                                       #
#                                                               #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see http://vina.scripps.edu for more information.      #
#################################################################

Detected 20 CPUs
Reading input ... done.
Setting up the scoring function ... done.
Analyzing the binding site ... done.
Using random seed: 68

### Parse output file

since each output have 9 modes, we will only take the 1st one for the sake of simplicity

In [43]:
output_dir = 'docking/docking_results'

energy_values = []

for result in os.listdir(output_dir):
    if result.endswith('.pdbqt'):
        with open(os.path.join(output_dir, result), 'r') as f:
            lines = f.readlines()
            for i, line in enumerate(lines):
                if line.startswith('MODEL 1'):
                    energy_line = lines[i+1]
                    if 'REMARK VINA RESULT' in energy_line:
                        val = float(energy_line.split()[3])
                        energy_values.append([result.replace('_out.pdbqt', ''), val])
                        
energy_values.sort(key=lambda x: x[1])

print(f'{"ligand":<15}{"energy(kcal/mol)"}')
print('-' * 30)
for entry in energy_values:
    print(f'{entry[0]:<15}{entry[1]:>5.1f}')

ligand         energy(kcal/mol)
------------------------------
ligand_8        -7.4
ligand_16       -7.1
ligand_11       -6.7
ligand_15       -6.6
ligand_17       -6.4
ligand_2        -6.3
ligand_5        -6.2
ligand_9        -6.0
ligand_18       -5.9
ligand_1        -5.9
ligand_19       -5.7
ligand_12       -5.5
ligand_3        -5.5
ligand_4        -5.5
ligand_10       -5.2
ligand_13       -5.1
ligand_7        -5.1
ligand_14       -4.8
ligand_0        -4.6
ligand_6        -3.4


# Pharmacophore modeling


### Pharmacophore modeling of ataxin2

In [ ]:
original_ligand

In [45]:
ataxin2 = original_ligand
ataxin2.write('docking/ataxin2.pdb')

In [51]:
from rdkit.Chem import AllChem, Descriptors
import py3Dmol

with open('docking/ataxin2.pdb', 'r') as f:
    pdb_block = f.read()

atx2_mol = Chem.MolFromPDBBlock(pdb_block)
atx2_mol = Chem.AddHs(atx2_mol)
AllChem.EmbedMolecule(atx2_mol)

logP = Descriptors.MolLogP(atx2_mol)
hba = Descriptors.NumHAcceptors(atx2_mol)
hbd = Descriptors.NumHDonors(atx2_mol)

aromatic_atoms=[]

for atom in atx2_mol.GetAtoms():
    if atom.GetIsAromatic():
        aromatic_atoms.append([atom.GetSymbol(), atom.GetIdx()])
        
atx2_feature = ['Ataxin 2', float(f'{logP:.3f}'), hba, hbd, len(aromatic_atoms)]

print("LogP(Hydrophobicity)", logP, flush=True)
print("Hydrogen Bond Acceptors", hba, flush=True)
print("Hydrogen Bond Donors", hbd, flush=True)
print("Aromatic atoms", len(aromatic_atoms), flush=True)

LogP(Hydrophobicity) -11.251729999999904
Hydrogen Bond Acceptors 29
Hydrogen Bond Donors 27
Aromatic atoms 12


In [53]:
print(atx2_mol.GetAromaticAtoms(), flush=True)

In [57]:
with open('docking/ataxin2.pdb', 'r') as f:
  pdb_content = f.read()

viewer = py3Dmol.view(width=400, height=400)
viewer.addModel(pdb_content, 'pdb')
viewer.setStyle({'stick':{},'sphere':{'radius':0.3}})
viewer.setBackgroundColor('0xeeeeee')

pharmacophoric_features = [('Hydrogen Donor', [1]), ('Hydrogen Acceptor', [2]),  ('Hydrophobic Interactions', [3])]

for feature, indices in pharmacophoric_features:
  if feature == 'Hydrogen Donor':
    viewer.setStyle({'model': 0, 'atom': {'serial': indices}}, {'sphere': {'color': 'blue', 'radius': 0.3}})
  if feature == 'Hydrogen Acceptor':
      viewer.setStyle({'model': 0, 'atom': {'serial': indices}}, {'sphere': {'color': 'red', 'radius': 0.3}})
  if feature == 'Hydrophobic Interactions':
      viewer.setStyle({'model': 0, 'atom': {'serial': indices}}, {'sphere': {'color': 'yellow', 'radius': 0.3}})


viewer.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

### Pharmacophore modeling of all ligands (n= 20)

In [ ]:
ligands_mols

In [65]:
ligand_features=[]

for i, ligand in enumerate(ligands_mols):
    ligand = Chem.AddHs(ligand)
    AllChem.EmbedMolecule(ligand)
    Chem.MolToPDBFile(ligand, f'docking/Ligands/ligand_{i}.pdb')
    
    logP = Descriptors.MolLogP(ligand)
    hba = Descriptors.NumHAcceptors(ligand)
    hbd = Descriptors.NumHDonors(ligand)

    aromatic_atoms=[]
    
    for atom in atx2_mol.GetAtoms():
        if atom.GetIsAromatic():
            aromatic_atoms.append([atom.GetSymbol(), atom.GetIdx()])
    
    ligand_features.append([f'ligand_{i}', float(f'{logP:.3f}'), hba, hbd, len(aromatic_atoms)])

[20:27:30] UFFTYPER: Unrecognized charge state for atom: 21


In [71]:
print('Pharmacophore comparison between ligands and original ligand')
print()
print(f'{"ligand name":<15}{"Hyprophobicity":<18}{"Hydrogen Acceptor":<21}{"Hydrogen Donor":<18}{"n Aromatic"}')
print('-'*105)
print(f'{atx2_feature[0]:<15}{atx2_feature[1]:<18}{atx2_feature[2]:<21}{atx2_feature[3]:<18}{atx2_feature[4]}')
for entry in ligand_features:
    print(f'{entry[0]:<15}{entry[1]:<18}{entry[2]:<21}{entry[3]:<18}{entry[4]}')  

Pharmacophore comparison between ligands and original ligand

ligand name    Hyprophobicity    Hydrogen Acceptor    Hydrogen Donor    n Aromatic
---------------------------------------------------------------------------------------------------------
Ataxin 2       -11.252           29                   27                12
ligand_0       3.22              1                    1                 12
ligand_1       5.015             2                    0                 12
ligand_2       4.676             2                    0                 12
ligand_3       4.631             3                    0                 12
ligand_4       4.147             2                    0                 12
ligand_5       2.567             6                    2                 12
ligand_6       1.015             3                    2                 12
ligand_7       4.603             6                    0                 12
ligand_8       3.809             5                    1                 12

Based on the pharmacophore features comparison, none of the chosen ligands have a high degree of similarity. Other than the number of aromatic atoms, all of the other features have big differences. This might be caused by the fact that the ligands were gathered by comparing similairty between the structure.

In [73]:
pdb_dir = 'docking/Ligands'

pdb_files = [f for f in os.listdir(pdb_dir) if f.endswith('.pdb')]

pdb_files = pdb_files[:20]

for pdb_file in pdb_files:
    with open(os.path.join(pdb_dir, pdb_file), 'r') as f:
        pdb_content = f.read()

    viewer = py3Dmol.view(width=250, height=250)
    viewer.addModel(pdb_content, 'pdb')
    viewer.setStyle({'stick': {}, 'sphere': {'radius': 0.3}})
    viewer.setBackgroundColor('0xeeeeee')

    pharmacophoric_features = [
        ('Hydrogen Donor', [1]),
        ('Hydrogen Acceptor', [2]),
        ('Hydrophobic Interactions', [3])
    ]

    for feature, indices in pharmacophoric_features:
        if feature == 'Hydrogen Donor':
            viewer.setStyle({'model': 0, 'atom': {'serial': indices}}, {'sphere': {'color': 'blue', 'radius': 0.3}})
        if feature == 'Hydrogen Acceptor':
            viewer.setStyle({'model': 0, 'atom': {'serial': indices}}, {'sphere': {'color': 'red', 'radius': 0.3}})
        if feature == 'Hydrophobic Interactions':
            viewer.setStyle({'model': 0, 'atom': {'serial': indices}}, {'sphere': {'color': 'yellow', 'radius': 0.3}})

    viewer.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

...

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Looking at the strucutre of each ligands also reveal that the overall strucutre of the chosen ligands were quite different from Ataxin 2

# QSAR

## Building QSAR dataset

QSAR Dataset will have docking energy, hydrophobicity, hydrogen donors, hydrogen acceptors, and n aromatic. Docking energy will be gathered from Model 1 of Autodock Vina result

In [ ]:
energy_values

In [75]:
energy_values_copy = energy_values.copy()
sorted_energy_values = sorted(energy_values_copy, key=lambda x: int(x[0].split('_')[1]))
sorted_energy_values

[['ligand_0', -4.6],
 ['ligand_1', -5.9],
 ['ligand_2', -6.3],
 ['ligand_3', -5.5],
 ['ligand_4', -5.5],
 ['ligand_5', -6.2],
 ['ligand_6', -3.4],
 ['ligand_7', -5.1],
 ['ligand_8', -7.4],
 ['ligand_9', -6.0],
 ['ligand_10', -5.2],
 ['ligand_11', -6.7],
 ['ligand_12', -5.5],
 ['ligand_13', -5.1],
 ['ligand_14', -4.8],
 ['ligand_15', -6.6],
 ['ligand_16', -7.1],
 ['ligand_17', -6.4],
 ['ligand_18', -5.9],
 ['ligand_19', -5.7]]

In [77]:
ligand_features_copy = ligand_features.copy()
sorted_ligand_feat = sorted(ligand_features_copy, key=lambda x: int(x[0].split('_')[1]))
sorted_ligand_feat

[['ligand_0', 3.22, 1, 1, 12],
 ['ligand_1', 5.015, 2, 0, 12],
 ['ligand_2', 4.676, 2, 0, 12],
 ['ligand_3', 4.631, 3, 0, 12],
 ['ligand_4', 4.147, 2, 0, 12],
 ['ligand_5', 2.567, 6, 2, 12],
 ['ligand_6', 1.015, 3, 2, 12],
 ['ligand_7', 4.603, 6, 0, 12],
 ['ligand_8', 3.809, 5, 1, 12],
 ['ligand_9', 4.901, 4, 0, 12],
 ['ligand_10', 5.341, 2, 1, 12],
 ['ligand_11', 3.52, 7, 1, 12],
 ['ligand_12', 2.55, 1, 1, 12],
 ['ligand_13', 4.631, 3, 1, 12],
 ['ligand_14', 2.322, 2, 2, 12],
 ['ligand_15', 7.016, 4, 0, 12],
 ['ligand_16', 3.383, 2, 0, 12],
 ['ligand_17', 4.448, 3, 2, 12],
 ['ligand_18', 8.644, 2, 1, 12],
 ['ligand_19', 2.525, 6, 0, 12]]

### Create a dataframe containing the combined data from energy_values and ligand_features

In [79]:
!pip install numpy scikit-learn pandas

In [81]:
import sklearn
import pandas as pd
import numpy as np

In [83]:
data = []

for i, entry in enumerate(ligand_features):
    data.append([entry[0], 
                 ligands_smls[i], 
                 sorted_energy_values[i][1],
                 entry[1],
                 entry[2],
                 entry[3],
                 entry[4]
                ])
    
data

[['ligand_0', 'CC(C)NC(C)Cc1ccc(I)cc1', -4.6, 3.22, 1, 1, 12],
 ['ligand_1', 'CN1CCCC(CC2c3ccccc3Sc3ccccc32)C1', -5.9, 5.015, 2, 0, 12],
 ['ligand_2', 'CN(C)CC/C=C1\\c2ccccc2CSc2ccccc21', -6.3, 4.676, 2, 0, 12],
 ['ligand_3', 'CN1CCCC(CN2c3ccccc3Sc3ccccc32)C1', -5.5, 4.631, 3, 0, 12],
 ['ligand_4', 'CC(C1=C(CCN(C)C)Cc2ccccc21)c1ccccn1', -5.5, 4.147, 2, 0, 12],
 ['ligand_5',
  'CCOC(=O)[C@H](CCc1ccccc1)N[C@@H](C)C(=O)N1Cc2ccccc2C[C@H]1C(=O)O',
  -6.2,
  2.567,
  6,
  2,
  12],
 ['ligand_6', 'S=c1[nH]cnc2nc[nH]c12', -3.4, 1.015, 3, 2, 12],
 ['ligand_7',
  'CCN(CCCCOC(=O)c1ccc(OC)c(OC)c1)C(C)Cc1ccc(OC)cc1',
  -5.1,
  4.603,
  6,
  0,
  12],
 ['ligand_8',
  'O=C1Cc2cc(CCN3CCN(c4nsc5ccccc45)CC3)c(Cl)cc2N1',
  -7.4,
  3.809,
  5,
  1,
  12],
 ['ligand_9',
  'CN1CCCCC1CCN1c2ccccc2Sc2ccc([S+](C)[O-])cc21',
  -6.0,
  4.901,
  4,
  0,
  12],
 ['ligand_10',
  'Cc1ccc(O)c([C@H](CCN(C(C)C)C(C)C)c2ccccc2)c1',
  -5.2,
  5.341,
  2,
  1,
  12],
 ['ligand_11',
  'CCCC(=O)O[C@]1(C(=O)COC(C)=O)CC[C@H]2[C

In [85]:
df = pd.DataFrame(data, columns=['Ligand Name', 'SMILES', 'Binding Energy', 'logP', 'H Donor', 'H Acceptor', 'n Aromatic'])
df

,Ligand Name,SMILES,Binding Energy,logP,H Donor,H Acceptor,n Aromatic
0,ligand_0,CC(C)NC(C)Cc1ccc(I)cc1,-4.6,3.220,1,1,12
1,ligand_1,CN1CCCC(CC2c3ccccc3Sc3ccccc32)C1,-5.9,5.015,2,0,12
2,ligand_2,CN(C)CC/C=C1\c2ccccc2CSc2ccccc21,-6.3,4.676,2,0,12
3,ligand_3,CN1CCCC(CN2c3ccccc3Sc3ccccc32)C1,-5.5,4.631,3,0,12
4,ligand_4,CC(C1=C(CCN(C)C)Cc2ccccc21)c1ccccn1,-5.5,4.147,2,0,12
5,ligand_5,CCOC(=O)[C@H](CCc1ccccc1)N[C@@H](C)C(=O)N1Cc2c...,-6.2,2.567,6,2,12
6,ligand_6,S=c1[nH]cnc2nc[nH]c12,-3.4,1.015,3,2,12
7,ligand_7,CCN(CCCCOC(=O)c1ccc(OC)c(OC)c1)C(C)Cc1ccc(OC)cc1,-5.1,4.603,6,0,12
8,ligand_8,O=C1Cc2cc(CCN3CCN(c4nsc5ccccc45)CC3)c(Cl)cc2N1,-7.4,3.809,5,1,12
9,ligand_9,CN1CCCCC1CCN1c2ccccc2Sc2ccc([S+](C)[O-])cc21,-6.0,4.901,4,0,12


### Predicting Binding Energy Based on Ligand Features

In [87]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

x = df[['logP', 'H Donor', 'H Acceptor', 'n Aromatic']]
y = df[['Binding Energy']]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'mse: {mse}')
print(f'R2: {r2}')

mse: 0.4081109552865368
R2: 0.3275205680139457
